##### Copyright 2018 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Load text with tf.data

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/alpha/tutorials/load_data/text"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This tutorial provides an example of how to use `tf.data.TextLineDataset` to load examples from text files. `TextLineDataset` is designed to create a dataset from a text file, in which each example is a line of text from the original file. This is potentially useful for any text data that is primarily line-based (for example, poetry or error logs).

In this tutorial, we'll use three different English translations of the same work, Homer's Illiad, and train a model to identify the translator given a single line of text.

## Setup

In [0]:
!pip install tensorflow==2.0.0-alpha0

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals 

import requests

import tensorflow as tf
import tensorflow_datasets as tfds


The texts of the three translations are by:

 - [William Cowper](https://en.wikipedia.org/wiki/William_Cowper) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)
 
 - [Edward, Earl of Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)
 
- [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29) — [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

The text files used in this tutorial have undergone some typical preprocessing tasks, mostly removing stuff — document header and footer, line numbers, chapter titles. Download these lightly munged files locally. 

In [0]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  r = requests.get("".join([DIRECTORY_URL, name]), allow_redirects=True)
  with open(name, 'wb') as f:
    f.write(r.content)

In [0]:
!ls

## Load text into datasets

Iterate through the files, loading each one into its own dataset.

Each example needs to be labeled individually labeled, so use `tf.data.Dataset.map` to apply a labeler function to each one. This will iterate over every example in the dataset, returning (`example, label`) pairs.

In [0]:
def get_labeler(label_int):
  """Returns a labeler function initialized with a specfic label."""
  label_tensor = tf.cast(label_int, tf.int64)

  def labeler(example):
    """Returns a labeled example."""
    return example, label_tensor
  
  return labeler

labeled_data_sets = []
for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(file_name)
  labeler_function = get_labeler(i)
  labeled_dataset = lines_dataset.map(labeler_function)
  labeled_data_sets.append(labeled_dataset)
  
  
  

Combine these labeled datasets into a single dataset, and shuffle it.


In [0]:
all_labeled_data = labeled_data_sets[0].concatenate(labeled_data_sets[1])
all_labeled_data = all_labeled_data.concatenate(labeled_data_sets[2])

all_labeled_data = all_labeled_data.shuffle(
                    tf.cast(50000, tf.int64), 
                    reshuffle_each_iteration=False)

You can use `tf.data.Dataset.take` and `print` to see what the `(example, label)` pairs look like. The `numpy` property shows each Tensor's value.

In [0]:
for ex in all_labeled_data.take(5):
  print(ex)

## Encode text lines as numbers

Machine learning models work on numbers, not words, so the string values need to be converted into lists of numbers. To do that, map each unique word to a unique integer.

### Build vocabulary

First, build a vocabulary by tokenizing the text into a collection of individual unique words. There are a few ways to do this in both TensorFlow and Python. For this tutorial:

1. Iterate over each example's `numpy` value.
2. Use `tfds.features.text.Tokenizer` to split it into tokens.
3. Collect these tokens into a Python set, to remove duplicates.
4. Get the size of the vocabulary for later use.

In [0]:
tokenizer = tfds.features.text.Tokenizer()
vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)


vocab_size = len(vocabulary_set)
vocab_size  

### Encode examples

Create an encoder by passing the `vocabulary_set` to `tfds.features.text.TokenTextEncoder`. The encoder's `encode` method takes in a string of text and returns a list of integers.

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

You can try this on a single line to see what the output looks like.

In [0]:
example_text = next(iter(all_labeled_data))[0].numpy()
example_text

In [0]:
encoded_example = encoder.encode(example_text)
encoded_example

Now run the encoder on the dataset by wrapping it in `tf.py_function` and  passing that to the dataset's `map` method.

In [0]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(text, label):
  return tf.py_function(
    encode, inp=[text, label], Tout=(tf.int64, tf.int64)
  )

all_encoded_data = all_labeled_data.map(encode_map_fn)

## Split the dataset into text and train batches

Use `tf.data.Dataset.take` and `tf.data.Dataset.skip` to create a small test dataset and a larger training set.

Before being passed into the model, the datasets need to be batched. Typically, the examples inside of a batch need to be the same size and shape. But, the examples in these datasets are not all the same size — each line of text had a different number of words. So use `tf.data.Dataset.padded_batch` (instead of `batch`) to pad the examples to the same size.

In [0]:
test_data = all_encoded_data.take(5000).padded_batch(50, padded_shapes=([-1],[]))
train_data = all_encoded_data.skip(5000).padded_batch(50, padded_shapes=([-1],[]))

Now, `test_data` and `train_data` are not collections of (`example, label`) pairs, but collections of batches. Each batch is a pair of (*many examples*, *many labels*) represented as arrays.

To illustrate:

In [0]:
one_batch = next(iter(test_data))


# a single text line, out of many in a batch
# notice the zero values padding out the list
one_batch[0].numpy()[0]

In [0]:
# a bunch of labels
one_batch[1]

Since we have introduced a new token encoding (the zero used for padding), the vocabulary size has increased by one.

In [0]:
vocab_size += 1

## Build the model



For simplicity's sake, start with a `tf.keras.Sequential` model.

In [0]:
model = tf.keras.Sequential()

The input is collection of integers representing text tokens (words). There are two downsides to this representation:

*  The integer-encoding is arbitrary. Therfore it does not capture any information about the words.

*  An integer-encoding can be challenging for a model to interpret. A linear classifier, for example, learns a single weight for each feature. Because different words may have a similar encoding, this feature-weight combination is not meaningful.

To overcome these challenges, the first layer of the model will be a `tf.keras.laters.Embedding` layer. This transforms each integer into a dense collection of floating point values. The exact encoding of integers to collections of floats is learned during training, so the encodings become meaningful to the model, rather than arbitrary integer assignments.

When creating an embedding layer, the two required arguments are:

*   input dimensionality — the number of integers appearing in the input set (that is, the size of the vocabulary)
*   embedding dimension — the number of floating point values that will represent a single integer (higher values discover more complex relationships between words, but require more data)

See the [Word Embeddings](../../tutorials/sequences/word_embeddings) tutorial for more details.

In [0]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

Next, we'll create a bidirectional [Long Short-Term Memory](https://en.wikipedia.org/wiki/Long_short-term_memory) layer by wrapping `tf.keras.layers.LSTM` in `tf.keras.layers.Birectional`. An LSTM is a type of Recurrent Neural Network that allows the model to understand data points in relationship to the datapoints that came before it. That is, a word in the context of a sentence, rather than a word in isolation.

To see the effect of the LSTM, you can try building the model with and without it, or try replacing it with an additional dense layer. 

In [0]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

Finally we'll have a series of one or more densely connected layers, with the last one being the output layer.

The output layer produces a one-hot encoding of one of three values (the labels representing the three different translators). Use the `softmax` activation to output three probability values. The node with the highest probability is the models prediction of an example's label.



In [0]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation=tf.keras.backend.relu))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3, activation='softmax'))

Finally, compile the model. For a softmax categorization model, use `sparse_categorical_crossentropy` as the loss function. You can try other optimizers, but `adam` is very common.

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Train the model

This model running on this data produces decent results (about 83%) after just two epochs and levels off quickly. A third epoch lowers accuracy by a tiny amount.

If you edit the model, try running several epochs to see when improvement levels off and overfitting sets in. To check progress after each epoch, validate against the `test_data` set. (This is the same as running `model.evaluate(test_data)` after each epoch.)

In [0]:
model.fit(train_data, epochs=3, validation_data=test_data)

In [0]:
model.evaluate(test_data)